In [3]:
!pip install -q --upgrade pip
!pip install -q mediapipe-model-maker

In [4]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import text_classifier

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [15]:
data_path = tf.keras.utils.get_file(
    fname='reproduce_4588.zip',
    origin='https://github.com/twiceyuan/twiceyuan/raw/main/reproduce_4588.zip',
    extract=True)
data_dir = os.path.join(os.path.dirname(data_path), 'reproduce_4588')  # folder name

18519/18519 [==============================] - 0s 0us/step


In [16]:
csv_params = text_classifier.CSVParams(
    text_column='sentence', label_column='label', delimiter=',')
train_data = text_classifier.Dataset.from_csv(
    filename=os.path.join(os.path.join(data_dir, 'train.csv')),
    csv_params=csv_params)
validation_data = text_classifier.Dataset.from_csv(
    filename=os.path.join(os.path.join(data_dir, 'dev.csv')),
    csv_params=csv_params)

In [17]:
supported_model = text_classifier.SupportedModels.AVERAGE_WORD_EMBEDDING_CLASSIFIER
hparams = text_classifier.HParams(epochs=10, batch_size=32, learning_rate=1, export_dir="awe_exported_models")
options = text_classifier.TextClassifierOptions(supported_model=supported_model, hparams=hparams)

In [18]:
model = text_classifier.TextClassifier.create(train_data, validation_data, options)

Epoch 1/10
30/30 [==============================] - 8s 118ms/step - loss: 1.3869 - accuracy: 0.2531 - val_loss: 1.3872 - val_accuracy: 0.2500
Epoch 2/10
30/30 [==============================] - 0s 7ms/step - loss: 1.3859 - accuracy: 0.2719 - val_loss: 1.3872 - val_accuracy: 0.2500
Epoch 3/10
30/30 [==============================] - 0s 8ms/step - loss: 1.3860 - accuracy: 0.2531 - val_loss: 1.3870 - val_accuracy: 0.2583
Epoch 4/10
30/30 [==============================] - 0s 7ms/step - loss: 1.3858 - accuracy: 0.2646 - val_loss: 1.3868 - val_accuracy: 0.2208
Epoch 5/10
30/30 [==============================] - 0s 9ms/step - loss: 1.3853 - accuracy: 0.2667 - val_loss: 1.3867 - val_accuracy: 0.2208
Epoch 6/10
30/30 [==============================] - 0s 9ms/step - loss: 1.3846 - accuracy: 0.2646 - val_loss: 1.3866 - val_accuracy: 0.2208
Epoch 7/10
30/30 [==============================] - 0s 7ms/step - loss: 1.3849 - accuracy: 0.2688 - val_loss: 1.3861 - val_accuracy: 0.2208
Epoch 8/10
30/30 [

In [19]:
loss, acc = model.evaluate(validation_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

8/8 [==============================] - 0s 3ms/step - loss: 1.3837 - accuracy: 0.2208
Test loss:1.3837242126464844, Test accuracy:0.22083333134651184


In [20]:
model.export_model()
model.export_labels(export_dir=options.hparams.export_dir)

In [24]:
supported_model = text_classifier.SupportedModels.MOBILEBERT_CLASSIFIER
hparams = text_classifier.HParams(epochs=10, batch_size=48, learning_rate=3e-5, export_dir="bert_exported_models")
options = text_classifier.TextClassifierOptions(supported_model=supported_model, hparams=hparams)

In [25]:
bert_model = text_classifier.TextClassifier.create(train_data, validation_data, options)

Using existing files at /tmp/model_maker/text_classifier/mobilebert_tiny
Using existing files at /tmp/model_maker/text_classifier/mobilebert_tiny
Epoch 1/10
20/20 [==============================] - 202s 665ms/step - loss: 13999.6689 - test_accuracy: 0.2562 - val_loss: 170.0739 - val_test_accuracy: 0.2333
Epoch 2/10
20/20 [==============================] - 8s 400ms/step - loss: 202.6890 - test_accuracy: 0.2448 - val_loss: 1.4850 - val_test_accuracy: 0.2833
Epoch 3/10
20/20 [==============================] - 9s 429ms/step - loss: 1.6946 - test_accuracy: 0.5719 - val_loss: 0.4492 - val_test_accuracy: 0.9625
Epoch 4/10
20/20 [==============================] - 9s 432ms/step - loss: 0.2046 - test_accuracy: 0.9635 - val_loss: 0.0205 - val_test_accuracy: 1.0000
Epoch 5/10
20/20 [==============================] - 13s 648ms/step - loss: 0.0243 - test_accuracy: 0.9906 - val_loss: 0.0029 - val_test_accuracy: 1.0000
Epoch 6/10
20/20 [==============================] - 8s 423ms/step - loss: 0.0041 - 

In [ ]:
loss, acc = bert_model.evaluate(validation_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

28/28 [==============================] - 2s 77ms/step - loss: 0.3878 - test_accuracy: 0.9071
Test loss:0.3878059685230255, Test accuracy:0.9071100950241089


In [26]:
from mediapipe_model_maker import quantization
quantization_config = quantization.QuantizationConfig.for_dynamic()
bert_model.export_model(quantization_config=quantization_config)
bert_model.export_labels(export_dir=options.hparams.export_dir)

ValueError: ignored